In [1]:
import pandas as pd
import sklearn.metrics
import numpy as np
import dit
from functools import wraps

In [2]:
def results_matrix(x, y):
    res = pd.DataFrame(index=sorted(x.unique()), columns=sorted(y.unique()))
    for i in res.index:
        res.loc[i,:] = y[(x[x == i]).index].value_counts(sort=False).sort_index()
    return res.fillna(0)

In [3]:
def van_dongen(x, y):
    m = results_matrix(x, y)
    n = m.sum().sum()
    sum1 = np.sum((m.loc[i, :].max() for i in m.index))
    sum2 = np.sum((m.loc[:, i].max() for i in m.columns))
    return 2 * n - sum1 - sum2

In [4]:
def variation_of_information(x, y):
    m = results_matrix(x, y)
    pmf = []
    outs = []
    n = m.sum().sum()
    for i in m.index:
        for j in m.columns:
            p_i_j = m.loc[i, j] / n
            outs.append((i, j))
            pmf.append(p_i_j)
    dist = dit.Distribution(outs, pmf=pmf)
    x_ent = dit.shannon.entropy(dist, rvs=[0])
    y_ent = dit.shannon.entropy(dist, rvs=[1])
    mutual_info = dit.shannon.mutual_information(dist, rvs_X=[0], rvs_Y=[1])
    return x_ent + y_ent - 2 * mutual_info

In [5]:
def map_score(x, y):
    m = results_matrix(x, y)
    left_score = 0
    for i in m.index:
        max_idx = m.loc[i, :].idxmax()
        left_score += (np.sum(m.loc[i, :]) - m.loc[i, max_idx]) / m.loc[i, max_idx]
    left_score = left_score / m.shape[0] / (m.shape[1] - 1) / 2
    
    right_score = 0
    for i in m.columns:
        max_idx = m.loc[:,i].idxmax()
        right_score += (np.sum(m.loc[:, i]) - m.loc[max_idx, i]) / m.loc[max_idx, i]
    right_score = right_score / m.shape[1] / (m.shape[0] - 1) / 2
    return left_score + right_score

In [6]:
MEASURE_NAMES = ['adjusted_rand_score', 
            'adjusted_mutual_info_score', 
            'normalized_mutual_info_score',
            'fowlkes_mallows_score',
            'jaccard_similarity_score',
           ]
MEASURES = list(map(lambda x: getattr(sklearn.metrics, x), MEASURE_NAMES)) + \
           [van_dongen, variation_of_information, map_score]

In [7]:
clusters = pd.read_csv('../01-cluster-sc01-sc02/SC01_assgn.csv')
clusters.columns = ['Cell', 'Cluster']
clusters.set_index('Cell', inplace=True)

In [8]:
clusters.Cluster.value_counts()

0     1255
1      819
2      789
3      721
4      445
5      410
6      311
7      281
8      272
9      214
10     202
11     197
12     186
13     177
14     165
15      89
16      78
17      55
Name: Cluster, dtype: int64

In [9]:
def frac_size(sample, frac):
    return len(sample.sample(frac=frac))

In [10]:
def on_some(func, selector):
    @wraps(func)
    def inner(clusters, frac=0.1):
        exp = clusters.copy()
        idx = exp[selector(exp)].sample(n=frac_size(clusters, frac)).index
        return func(exp, idx)
    return inner

def on_all(func):
    return on_some(func, lambda exp: exp.index)

def on_big(func):
    return on_some(func, lambda exp: exp < BIG)

def on_small(func):
    return on_some(func, lambda exp: exp > SMALL)

In [11]:
@on_all
def rename(exp, idx):
    exp[exp.index] += 18
    return exp

In [12]:
@on_all
def mistakes_all(exp, idx):
    exp[idx] -= 1
    exp[exp == -1] = 17
    return exp

In [13]:
@on_big
def mistakes_big(exp, idx):
    exp[idx] -= 1
    exp[exp == -1] = 17
    return exp

In [14]:
@on_small
def mistakes_small(exp, idx):
    exp[idx] -= 1
    return exp

In [15]:
@on_all
def splits_all(exp, idx):
    exp[idx] += 18
    return exp

In [16]:
@on_big
def splits_big(exp, idx):
    exp[idx] += 18
    return exp

In [17]:
@on_small
def splits_small(exp, idx):
    exp[idx] += 18
    return exp

In [18]:
@on_all
def unites_all(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 2 == 1)] -= 1
    return exp

In [19]:
@on_big
def unites_big(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 2 == 1)] -= 1
    return exp

In [20]:
@on_small
def unites_small(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 2 == 1)] -= 1
    return exp

In [21]:
@on_all
def splits_unites_all(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 4).isin((1,))] -= 1
    return exp

In [22]:
@on_big
def splits_unites_big(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 4).isin((1,))] -= 1
    return exp

In [23]:
@on_small
def splits_unites_small(exp, idx):
    exp[idx] += 18
    exp[(exp > 17) & (exp % 4).isin((1,))] -= 1
    return exp

In [24]:
EXPERIMENTS = [
    'rename',
    'mistakes_all',
    'mistakes_big',
    'mistakes_small',
    'splits_all',
    'splits_big',
    'splits_small',
    'unites_all',
    'unites_big',
    'unites_small',
    'splits_unites_all',
    'splits_unites_big',
    'splits_unites_small',
]

In [25]:
BIG = 4
SMALL = 9
for key in EXPERIMENTS:
    clusters[key] = locals()[key](clusters.Cluster, frac=0.1)

In [26]:
results = pd.DataFrame(index=[x.__name__ for x in MEASURES])
for key in EXPERIMENTS:
    res = []
    for measure in MEASURES:
        res.append(measure(clusters.Cluster, clusters[key]))
    results[key] = res

In [27]:
results

,rename,mistakes_all,mistakes_big,mistakes_small,splits_all,splits_big,splits_small,unites_all,unites_big,unites_small,splits_unites_all,splits_unites_big,splits_unites_small
adjusted_rand_score,1.0,0.818480,0.766572,0.971127,0.892631,0.850640,0.987858,0.885222,0.835223,0.980030,0.889731,0.844269,0.983129
adjusted_mutual_info_score,1.0,0.874564,0.901472,0.947082,0.886708,0.908236,0.956129,0.884372,0.906053,0.955236,0.885610,0.907283,0.955517
normalized_mutual_info_score,1.0,0.880436,0.912824,0.950925,0.942604,0.953488,0.978100,0.928647,0.939876,0.964681,0.936515,0.947609,0.970574
fowlkes_mallows_score,1.0,0.835602,0.788575,0.973900,0.906232,0.871143,0.989062,0.899118,0.855434,0.981932,0.903452,0.864760,0.984747
jaccard_similarity_score,0.0,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940,0.899940
van_dongen,0.0,1265.000000,1143.000000,1032.000000,667.000000,667.000000,482.000000,958.000000,935.000000,772.000000,803.000000,802.000000,642.000000
variation_of_information,0.0,0.895674,0.657490,0.364329,0.467421,0.372256,0.168663,0.565973,0.469096,0.265903,0.510686,0.414272,0.223403
map_score,0.0,0.006659,0.002757,0.019440,0.001580,0.001218,0.006450,0.009963,0.003386,0.011514,0.004881,0.002580,0.008737
